# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,295.29,88,100,10.75,PN,1721366106
1,1,rehoboth,-23.3167,17.0833,278.82,36,0,2.33,NaN,1721366128
2,2,chifeng,42.2683,118.9636,308.90,20,45,2.55,CN,1721366151
3,3,grytviken,-54.2811,-36.5092,271.92,94,79,2.21,GS,1721366174
4,4,port-aux-francais,-49.3500,70.2167,273.80,71,95,9.55,TF,1721366196


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot with hvplot points
# Configure the map plot_3
map_plot_3 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Population",
    scale = 0.01,
    color = "City"
)

# Display the map
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values


# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,295.29,88,100,10.75,PN,1721366106
2,2,chifeng,42.2683,118.9636,308.90,20,45,2.55,CN,1721366151
3,3,grytviken,-54.2811,-36.5092,271.92,94,79,2.21,GS,1721366174
4,4,port-aux-francais,-49.3500,70.2167,273.80,71,95,9.55,TF,1721366196
5,5,ulsteinvik,62.3432,5.8487,288.75,88,98,1.79,NO,1721366219


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,88,
2,chifeng,CN,42.2683,118.9636,20,
3,grytviken,GS,-54.2811,-36.5092,94,
4,port-aux-francais,TF,-49.3500,70.2167,71,
5,ulsteinvik,NO,62.3432,5.8487,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "bias": "",
    "categories": "",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]


    # Add the current city's latitude and longitude to the params dictionary
    params["bias"] = f"proximity:{lat},{lng}"
    params["categories"] = "accommodation.hotel"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)


    # Convert the API response to JSON format
    name_address = response.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found for " + lat + ", " + lng

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
<Response [200]>
adamstown - nearest hotel: No hotel found for https://api.geoapify.com/v2/places?bias=proximity%3A-25.066%2C-130.1015&categories=accommodation.hotel&apiKey=cf73032aa1364b7491d9bd240d5a73ea
<Response [200]>
chifeng - nearest hotel: No hotel found for https://api.geoapify.com/v2/places?bias=proximity%3A42.2683%2C118.9636&categories=accommodation.hotel&apiKey=cf73032aa1364b7491d9bd240d5a73ea
<Response [200]>
grytviken - nearest hotel: The Smalleast Hotel
<Response [200]>
port-aux-francais - nearest hotel: Cafémma Hotel
<Response [200]>
ulsteinvik - nearest hotel: Casa Mia
<Response [200]>
lagunas - nearest hotel: No hotel found for https://api.geoapify.com/v2/places?bias=proximity%3A-5.2269%2C-75.6753&categories=accommodation.hotel&apiKey=cf73032aa1364b7491d9bd240d5a73ea
<Response [200]>
isafjordur - nearest hotel: La Belle Rodriguaise
<Response [200]>
talnakh - nearest hotel: No hotel found for https://api.geoapify.com/v2/places?bias=proximity%3A69.

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,88,No hotel found for https://api.geoapify.com/v2...
2,chifeng,CN,42.2683,118.9636,20,No hotel found for https://api.geoapify.com/v2...
3,grytviken,GS,-54.2811,-36.5092,94,The Smalleast Hotel
4,port-aux-francais,TF,-49.3500,70.2167,71,Cafémma Hotel
5,ulsteinvik,NO,62.3432,5.8487,88,Casa Mia
6,lagunas,PE,-5.2269,-75.6753,75,No hotel found for https://api.geoapify.com/v2...
7,isafjordur,IS,66.0755,-23.1240,96,La Belle Rodriguaise
8,talnakh,RU,69.4865,88.3972,41,No hotel found for https://api.geoapify.com/v2...
9,gondar,ET,12.6000,37.4667,83,Kartibubbo Village
10,barra de santo antonio,BR,-9.4047,-35.5072,80,Bertrand's Cottage


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_4 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "Hotel Name"
)

# Display the map
map_plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)